In [1]:
from openai import OpenAI

In [2]:
client = OpenAI(
    base_url = 'http://localhost:11434/v1/',
    api_key = 'ollama',
)

In [3]:
response = client.chat.completions.create(
    model="phi3",
    messages=[{"role":"user", "content":"Is it too late to join the course?"}]
)

In [5]:
print(response.choices[0].message.content)

 Whether or not it's "too late" to join a course depends on several factors:

1. Course Policy: Check the enrollment policies of the specific course you are interested in. Some courses may have open registration periods, while others might close after set dates or at certain levels of prerequisite completion (e.g., only accepting students who have completed a particular foundational class).

2. Course Availability: Certain online courses and distance-learning programs are often available throughout the year without strict enrollment periods, while some in-person classes may follow traditional academic terms with defined start and end dates.

3. Waitlist Position: If you're trying to join a class that has reached its maximum capacity, it is still possible to join by placing yourself on the waitlist provided by instructors or program coordinators. The likelihood of being accepted into an available spot will vary based on various factors, such as course demand and overall enrollment numbe